In [ ]:
!bzip2 -d svt-2023.xml.bz2


In [ ]:
!pip3.10 uninstall spacy
# grep -o "<sentence" svt-2023.xml | wc -l


Found existing installation: spacy 3.8.4
Uninstalling spacy-3.8.4:
  Would remove:
    /Library/Frameworks/Python.framework/Versions/3.10/bin/spacy
    /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy-3.8.4.dist-info/*
    /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/spacy/*
Proceed (Y/n)? ^C


In [1]:
import xml.etree.ElementTree as ET

xml_file = "svt-2023.xml"

count = 0
for event, elem in ET.iterparse(xml_file, events=("end",)):
    if elem.tag == "sentence":
        count += 1
        elem.clear()  # Free memory

print(f"Total sentences in the file: {count}")


KeyboardInterrupt: 

In [ ]:
import xml.etree.ElementTree as ET
import csv
from transformers import pipeline
import re

# Load Swedish BERT NER model
nlp = pipeline("ner", model="KBLab/bert-base-swedish-cased-ner", tokenizer="KBLab/bert-base-swedish-cased-ner")

# Define input/output files
xml_file = "svt-2023.xml"
csv_file = "sentences_entities_bert.csv"

# Helper function to extract capitalized words as potential names
def extract_capitalized_words(sentence, found_entities):
    words = sentence.split()
    possible_names = []
    for i, word in enumerate(words):
        # If the word starts with an uppercase letter and is not the first word (to avoid sentence-start capitalization)
        if re.match(r"^[A-ZÅÄÖ][a-zåäö]+$", word) and i > 0:
            if word not in found_entities and len(word) > 3:
                print(f"Found possible name: {word}")
                possible_names.append(word)
    return possible_names

def clean_sentence(sentence):
    # Remove unwanted symbols but keep .,!? (normal punctuation)
    sentence = re.sub(r'[-:…]', ' ', sentence)  # Replace dashes and colons with space
    sentence = re.sub(r'\[.*?\]', '', sentence)  # Remove text inside [...]
    
    # Normalize spaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

# Open CSV file for writing
with open(csv_file, "w", encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["sentence", "entities"])  # CSV headers

    # Parse XML
    context = ET.iterparse(xml_file, events=("start", "end"))
    count = 0

    for event, elem in context:
        if event == "end" and elem.tag == "sentence":
            # Extract sentence text
            words = [token.text for token in elem.findall(".//token") if token.text]
            sentence = " ".join(words)
            
            

            # Run BERT NER model
            bert_entities = []
            current_entity = ""
            current_label = None

            for token in nlp(sentence):
                word = token["word"]
                label = token["entity"]
                score = token["score"]

                if word.startswith("##"):  
                    # Append subword parts correctly
                    current_entity += word[2:]
                else:
                    if current_entity and current_label in ["PER", "ORG", "LOC", "EVN"]:
                        bert_entities.append(current_entity)  # Store completed entity

                    # Start a new entity only if confidence is high
                    if score > 0.95 and label in ["PER", "ORG", "LOC", "EVN"]:
                        current_entity = word
                        current_label = label
                    else:
                        current_entity = ""
                        current_label = None

            if current_entity and current_label in ["PER", "ORG", "LOC", "EVN"]:
                bert_entities.append(current_entity)  # Add the last entity

            # Write to CSV
            sentence = clean_sentence(sentence)
            writer.writerow([sentence, ", ".join(all_entities)])

            # Free memory
            elem.clear()

            count += 1
            if count >= 50:  # Limit to 50 sentences for testing
                break

print(f"✅ Processed first 50 sentences with BERT! Saved as {csv_file}.")


Some weights of the model checkpoint at KBLab/bert-base-swedish-cased-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Additional names found: []
Additional names found: []
Found possible name: Jwan
Found possible name: Ali
Additional names found: ['Jwan', 'Ali']
Found possible name: Det
Additional names found: ['Det']
Found possible name: Jacob
Found possible name: Lagercrantz
Additional names found: ['Jacob', 'Lagercrantz']
Additional names found: []
Found possible name: Det
Additional names found: ['Det']
Additional names found: []
Found possible name: Fredriksson
Additional names found: ['Fredriksson']
Found possible name: Där
Found possible name: Henrik
Found possible name: Johansson
Additional names found: ['Där', 'Henrik', 'Johansson']
Found possible name: Polenfärjan
Found possible name: Stor
Additional names found: ['Polenfärjan', 'Stor']
Found possible name: Karlshamnspiloten
Additional names found: ['Karlshamnspiloten']
Additional names found: []
Additional names found: []
Additional names found: []
Found possible name: Det
Additional names found: ['Det']
Additional names found: []
Additiona

In [5]:
from transformers import pipeline

nlp = pipeline("ner", model="KBLab/bert-base-swedish-cased-ner", tokenizer="KBLab/bert-base-swedish-cased-ner")

test_sentence = "Under lördagsförmiddagen kom ägaren Jwan Ali till platsen ."

# Run NER
tokens = nlp(test_sentence)

# Print results
for token in tokens:
    print(token)


Some weights of the model checkpoint at KBLab/bert-base-swedish-cased-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'entity': 'TME', 'score': 0.99950325, 'index': 1, 'word': 'Under', 'start': 0, 'end': 5}
{'entity': 'TME', 'score': 0.9993074, 'index': 2, 'word': 'lördags', 'start': 6, 'end': 13}
{'entity': 'TME', 'score': 0.9996593, 'index': 3, 'word': '##för', 'start': 13, 'end': 16}
{'entity': 'TME', 'score': 0.99969447, 'index': 4, 'word': '##middagen', 'start': 16, 'end': 24}
{'entity': 'PER', 'score': 0.99985874, 'index': 7, 'word': 'J', 'start': 36, 'end': 37}
{'entity': 'PER', 'score': 0.99984276, 'index': 8, 'word': '##wan', 'start': 37, 'end': 40}
{'entity': 'PER', 'score': 0.9997129, 'index': 9, 'word': 'Ali', 'start': 41, 'end': 44}
